In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def save_model(epoch):

  path = f"./model_epoch_{epoch}.pth"
  torch.save(model.state_dict(), path)

In [ ]:
path = '/content/drive/MyDrive/cropped/cropped'

In [ ]:
import copy

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from PIL import Image
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import numpy as np
import os
import re

In [ ]:
def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything(42)


#train_labels
csv_file_path = 'labels.csv'
# And corresponding images
images_path = path


# Read the CSV file
df = pd.read_csv(csv_file_path)



In [ ]:
print(df.iloc[2])

id                        2
label                     1
source_img    train_673.png
Name: 2, dtype: object


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]

        img_path = f'{self.image_dir}/{img_name}.png'
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



train_df, test_df = train_test_split(df, test_size=0.1)

# Create the dataset
train_dataset = CustomDataset(dataframe=train_df, image_dir=images_path, transform=transform)
test_dataset = CustomDataset(dataframe=test_df, image_dir=images_path, transform=transform)


# Create the DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)



In [ ]:
import torch.nn as nn
from torchvision import models

num_classes = 4

model = models.vgg16(pretrained=True)
# Freeze the features layers (optional, depending on your needs)
for param in model.features.parameters():
    param.requires_grad = False

# Replace the classifier
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 137MB/s]


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
num_epochs = 10






def validation_metrics(model, dataloader, device, criterion):
    model.eval()
    validation_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_validation_loss = validation_loss / len(dataloader)
    accuracy = 100 * correct / total
    return accuracy, avg_validation_loss
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    training_accuracy = 100 * correct / total

    validation_accuracy, validation_loss = validation_metrics(model, test_loader, device, criterion)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Training Accuracy: {training_accuracy:.2f}%')
    print(f'Validation Accuracy: {validation_accuracy:.2f}%, Validation Loss: {validation_loss:.4f}')

    save_model(epoch)

Epoch 1/10, Loss: 0.9023, Training Accuracy: 67.47%
Validation Accuracy: 72.24%, Validation Loss: 0.7378
Epoch 2/10, Loss: 0.7473, Training Accuracy: 72.73%
Validation Accuracy: 75.07%, Validation Loss: 0.6694
Epoch 3/10, Loss: 0.6810, Training Accuracy: 74.28%
Validation Accuracy: 74.79%, Validation Loss: 0.6335
Epoch 4/10, Loss: 0.6283, Training Accuracy: 75.63%
Validation Accuracy: 76.77%, Validation Loss: 0.6096
Epoch 5/10, Loss: 0.5822, Training Accuracy: 78.09%
Validation Accuracy: 76.77%, Validation Loss: 0.5929
Epoch 6/10, Loss: 0.5452, Training Accuracy: 78.90%
Validation Accuracy: 76.77%, Validation Loss: 0.5965
Epoch 7/10, Loss: 0.4994, Training Accuracy: 79.85%
Validation Accuracy: 77.05%, Validation Loss: 0.5743
Epoch 8/10, Loss: 0.4592, Training Accuracy: 81.71%
Validation Accuracy: 76.77%, Validation Loss: 0.5912
Epoch 9/10, Loss: 0.4094, Training Accuracy: 83.88%
Validation Accuracy: 72.52%, Validation Loss: 0.6109
Epoch 10/10, Loss: 0.3660, Training Accuracy: 86.37%
Va